##### Formulation of the GAMS model in JuMP from 
###### AN IMPROVED FORMULATION FOR A GAS TRANSPORTATION PROBLEM  (ERWIN KALVELAGEN)
###### https://www.researchgate.net/publication/2568197_An_Improved_Formulation_For_A_Gas_Transportation_Problem

##### Further information on JuMP: 
https://arxiv.org/pdf/1508.01982.pdf

# Read data and Data cleaning

In [48]:
using DataFrames
using JuMP
using ExcelReaders

In [58]:
Ndata = readtable("Ndata.csv", separator=';') # node data
head(Ndata)

,i,slo,sup,plo,pup,c
1,Anderlues,0.0,1.2,0,66.2,0.0
2,Antwerpen,-Inf,-4.034,30,80.0,0.0
3,Arlon,-Inf,-0.222,0,66.2,0.0
4,Berneau,0.0,0.0,0,66.2,0.0
5,Blaregnies,-Inf,-15.616,50,66.2,0.0
6,Brugge,-Inf,-3.918,30,80.0,0.0


In [59]:
Adata = readtable("Adata.csv", separator=';') # line data
head(Adata)

,a,i,j,D,L,act
1,1,Zeebrugge,Dudzele,890.0,4.0,0
2,2,Zeebrugge,Dudzele,890.0,4.0,0
3,3,Dudzele,Brugge,890.0,6.0,0
4,4,Dudzele,Brugge,890.0,6.0,0
5,5,Brugge,Zomergem,890.0,26.0,0
6,6,Loenhout,Antwerpen,590.1,43.0,0


In [51]:
I = Ndata[:i]; #set of cities -->  i,j element of I 

In [52]:
A = length(Adata[:a])

24

## Constants

In [60]:
T= 281.5
e = 0.05
den = 0.616
z = 0.8
epsilon = 0.05
delta = 0.6106;

In [54]:
#readxl(DataFrame,"Adata.xlsx","Tabelle1!A1:F25");

## Read line data
### Calculate pipeline characteristics

$$ lambda_{aij} = \frac {1} { \frac {\sqrt{2* \log_{10}*3.7*diameter_{aij}}} {epsilon}}  $$

$$ pipecoeff_{aij} = 96.074830*10^{-15} * \frac {diameter_{aij}^5 } { lambda_{aij}*z*T*length_{aij}*delta) } $$

In [55]:
head(Adata)

,a,i,j,D,L,act
1,1,Zeebrugge,Dudzele,890.0,4.0,0
2,2,Zeebrugge,Dudzele,890.0,4.0,0
3,3,Dudzele,Brugge,890.0,6.0,0
4,4,Dudzele,Brugge,890.0,6.0,0
5,5,Brugge,Zomergem,890.0,26.0,0
6,6,Loenhout,Antwerpen,590.1,43.0,0


In [56]:
df = Adata
lambda = Dict()
pipecoeff = Dict()
act = Dict()

for itera in 1:size(df, 1)
    i = df[itera,2]
    j = df[itera,3]
    lambda[i,j] = 1/sqrt(2*log10(3.7*df[itera,4]/epsilon))
    pipecoeff[i,j] = 96.074830e-15 *  (df[itera,4]^5/(lambda[i,j]*z*T*df[itera,5]*delta) )
    println(tuple(df[itera,1:3]...))
    println(tuple(df[itera,:1:3]...))
    di[tuple(df[itera,:1:3]...)]  = df[i,4]

    len[tuple(df[itera,:1:3]...)] = df[i,5]

    act[tuple(df[itera,:1:3]...)] = df[itera,6]
end



LoadError: [91mMethodError: no method matching start(::DataFrames.DataFrame)[0m
Closest candidates are:
  start([91m::SimpleVector[39m) at essentials.jl:258
  start([91m::Base.MethodList[39m) at reflection.jl:560
  start([91m::ExponentialBackOff[39m) at error.jl:107
  ...[39m

In [57]:
# I = unique(df[:,2]);

In [ ]:
#parMatrix = readdlm("Adata.csv", ';',skipstart=1);
#df = parMatrix
#di = Dict() #diameter
#len = Dict() #length
#act = Dict() #active pipeline

#for i in 1:size(df, 1)
#    #println(tuple(df[i, :]...))
#    di[tuple(df[i,:1:3]...)]  = df[i,4]
#    len[tuple(df[i,:1:3]...)] = df[i,5]
#    act[tuple(df[i,:1:3]...)] = df[i,6]
#end

## Read node data

In [ ]:
df = Ndata
si = [] #supply subset
c = Dict()
plo = Dict()
slo = Dict()
sup = Dict()

for b in 1:size(df, 1)
    ival = df[b,1]
    c[ival] = df[b,6]
    plo[ival] = df[b,4]
    slo[ival] = df[b,2]
    sup[ival] = df[b,3]
    if df[b,6] > 0 
        push!(si,ival)
    end 

end

In [ ]:
J = Dict()
for itera in 1:size(df, 1)    
    ival = df[itera,2]
    J[ival] = df[df[:,2] .== ival, 3]
end


In [ ]:
for i in I
    try 
        J[i]
    catch
        J[i]= ""
    end    
        
end

## Edge formulation
###### https://arxiv.org/pdf/1508.01982.pdf p.6

In [85]:
immutable Edge
    from; to; cost; capacity
end

edges = [Edge(2,3,4,5),Edge(2,3,4,5)]
#f = Edge(("from","to", "cost","capactiy") => basename = [
#        Edge("1","2","3","4") for (a,b,c,d) in test]);

In [92]:
immutable Edge
    route; from; to; diameter; length; active
end

for itera in 1:size(df, 1)
    println(df[itera,:1])
    a = df[itera,:1] #set route
    i = df[itera,:2] #set start node
    j = df[itera,:3] #set end node
    d = df[itera,:4] #diameter
    l = df[itera,:5] #length
    act = df[itera,:6] #active? --> compressor
    push!(edges,Edge(a,i,j,d,l,act))
end

LoadError: [91minvalid redefinition of constant Edge[39m

# Model 

In [ ]:
using Gurobi #Solver

In [ ]:
m = Model(solver = GurobiSolver(Presolve=0))

## Sets and subsets

In [ ]:
head(Ndata)

In [ ]:
i_sup = Ndata[Ndata[:sup] .>0, :i] # supply nodes

In [ ]:
head(Adata)

### aij
$$aij $$ 

In [72]:
test = Adata #active

AIJ = Tuple[]

for i in 1:size(test, 1)
    print(tuple((test[i,1]),test[i,2],test[i,3]))
    push!(AIJ,tuple((test[i,1]),test[i,2],test[i,3]))
end

(1, "Zeebrugge", "Dudzele")(2, "Zeebrugge", "Dudzele")(3, "Dudzele", "Brugge")(4, "Dudzele", "Brugge")(5, "Brugge", "Zomergem")(6, "Loenhout", "Antwerpen")(7, "Antwerpen", "Gent")(8, "Gent", "Zomergem")(9, "Zomergem", "Peronnes")(10, "Voeren", "Berneau")(11, "Voeren", "Berneau")(12, "Berneau", "Liege")(13, "Berneau", "Liege")(14, "Liege", "Warnand")(15, "Liege", "Warnand")(16, "Warnand", "Namur")(17, "Namur", "Anderlues")(18, "Anderlues", "Peronnes")(19, "Peronnes", "Mons")(20, "Mons", "Blaregnies")(21, "Warnand", "Wanze")(22, "Wanze", "Sinsin")(23, "Sinsin", "Arlon")(24, "Arlon", "Petange")

### aij_active
$$aij_{active} $$

In [ ]:
test = Adata[Adata[:act] .== 1, :2:3] #active

a = Tuple[]
b = Tuple[]

for i in 1:size(test, 1)
    print(tuple((test[i,1]),test[i,2]))
    push!(a,tuple((test[i,1]),test[i,2]))
end

test = Adata[Adata[:act] .== 0, :2:3] #passive

for i in 1:size(test, 1)
    #println(Array(test[i,1:2]))
    push!(b,tuple((test[i,1]),test[i,2]))
end

In [ ]:
A = length(I)

## Variables

##### When using a tuple as an index (a,i,j) the automated latex generation does not really work

In [ ]:
# f_aij
f = Dict((a,i, j) => @variable(m, basename = 
        "x[$(string(a)),$(string(i)),$(string(j))]") for (a,i,j) in AIJ);

In [ ]:
#@variable(m, f[for a,b,c in AIJ] >= 0 ) # f_a,i,j 

In [ ]:
@variable(m, -Inf<=s[I]<=Inf  ) # s_a,i,j, supply demand

In [ ]:
@variable(m, pi[I] >= 0 ) #squared pressure

In [ ]:
@variable(m, sc >= 0 )

In [ ]:
m

## Objective

In [ ]:
@objective(m, Min, sum(s[i] * c[i] for i in si ) )

## Constraints


Equations 
* flowbalance(i)    flow conservation
* weymouthp(a,i,j)  flow pressure relationship - passive
* weymoutha(a,i,j)  flow pressure relationship - active
* defsc             definition of supply cost


In [ ]:
# node balance
for i in I
    @constraint(m, sum(f[aij] for aij in AIJ) == 
                sum(f[aij]  for aij in AIJ) + s[i])
end

In [ ]:
# passive linear
for ele in b
    i = ele[1]
    j = ele[2]
    @constraint(m, f[i, j] == pipecoeff[i, j] * (pi[i] - pi[j]) ) ;
end

In [ ]:
# active linear

for ele in a
    i = ele[1]
    j = ele[2]
    @constraint(m, f[i, j] >= pipecoeff[i, j] * (pi[i] - pi[j]) ) ;
end


In [ ]:
for i in I
    @constraint(m, s[i] >= slo[i] ) 
    @constraint(m, s[i] <= sup[i] ) 
    @constraint(m, pi[i] >= sqrt(plo[i]) ) 
end

## Solve

In [ ]:
status = solve(m)

In [ ]:
println("f = ", getvalue(pi))

# Garbage

In [ ]:
function fa{T<:Any}(A::Array{T,2})
    return tuple([A[c,:] for c in 1:size(A,1)]...)
end

A = rand(4,5)
tup = fa(A)